# Redis
### Store cars data

#### Install needed to run Redis

In [ ]:
#!pip install redis

In [ ]:
#!pip install flask-bootstrap

#### Preparing the data

In [58]:
import pandas as pd
import pickle

In [59]:
df_carList = pd.read_csv("./datasets/Car_listings.csv", delimiter=",")

We can see that this dataset has a Price column, which we don't need, beacuse this is a rental company where customers doesn't buy the cars. Because of this we remove the Price column:

In [61]:
df_carList = df_carList.drop(['Price'], axis = 1)
df_carList.head(1)

KeyError: "['Price'] not found in axis"

In [63]:
df_carList = df_carList.sample(frac = 1)

#### We need to convert the csv file to JSON, so that we can store the data in redis

In [64]:
df_carList.to_json('./datasets/cars.json', orient='split', compression='infer', index='true')

In [66]:
df = pd.read_json('./datasets/cars.json', orient ='split', compression = 'infer')

In [67]:
df

,Year,Mileage,City,State,Vin,Make,Model
400278,2016,24925,Tucson,AZ,1HGCR2F82GA002625,Honda,Accord
330804,2017,15630,Katy,TX,1FMJU1JT4HEA55066,Ford,ExpeditionXLT
836297,2015,29835,Wilminton,OH,3VWLA7AJ0FM239576,Volkswagen,Jetta
778225,2013,53100,San Antonio,TX,4T1BK1EB5DU009824,Toyota,AvalonXLE
539953,2017,6636,Ballwin,MO,3KPFK4A72HE061487,Kia,ForteLX
...,...,...,...,...,...,...,...
508098,2012,42510,Wappingers Falls,NY,1C4BJWCG4CL117326,Jeep,Wrangler4WD
511276,2012,87207,Watertown,WI,1C4BJWDG8CL102827,Jeep,Wrangler
793116,2012,90234,Knoxville,TN,5TDBK3EH5CS153140,Toyota,Highlander4WD
428259,2016,19,Grapevine,TX,5FNRL5H43GB168472,Honda,OdysseyEX


In [68]:
import redis
import json
import time

In [69]:
r = redis.Redis(db=1)
#r.flushdb()

In [70]:
with open('./datasets/cars.json') as json_file:
    data = json.load(json_file)
 
    # Print the type of data variable
    #print("Type:", type(data))
    
    #print("ITEMS: ", data.items())
      
    # Print the data of dictionary
    #print("\nData:", data['data'][0]['ID IPEDS Occupation Parent'])
    print("\nData:", data['data'][0])


Data: [2016, 24925, 'Tucson', ' AZ', '1HGCR2F82GA002625', 'Honda', 'Accord']


In [71]:
# Loop og give hver værdi en key som er deres kolonne navn
columns = list(df)
listOfDict = []

def smallDictFunction(co: [], di: {}) -> {}:
    smallDicts = {}
    for c in co:
        for d in di:
            if (columns.index(c) == dic.index(d)):
                #print(c + ': ' + str(d))
                #print(d)
                smallDicts[c] = d
        
    return smallDicts
   

for dic in data['data']:
    listOfDict.append(smallDictFunction(columns, dic))
    if (data['data'].index(dic)+1 == 200000):
        break    # break here


#print(test())
#print(smallDicts)
print(listOfDict)
#print(fullDict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [72]:
index_list = [*range(1, (len(listOfDict)+1), 1)]
len(index_list)

200000

In [73]:
zipped = zip(index_list, listOfDict)
fullDict = dict(zipped)
print(fullDict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [74]:
with open("./datasets/carsDict.json", "w") as write_file:
    json.dump(fullDict, write_file, indent=4)

In [75]:
with open('./datasets/carsDict.json') as json_file:
    data = json.load(json_file)

In [76]:
for dic in data:
    p_mydict = pickle.dumps(data[dic])
    r.set(dic,p_mydict)

In [90]:
read_dict = r.get(200000)
yourdict = pickle.loads(read_dict)
yourdict

{'Year': 2012,
 'Mileage': 104716,
 'City': 'Tempe',
 'State': ' AZ',
 'Vin': '1HGCP2F67CA001727',
 'Make': 'Honda',
 'Model': 'Accord'}

#### Postgres
Vi skal kunne søge i postgres efter en person, lave den person inclussiv addresse om til dict og smide den i mongo i samme dict som redis

In [92]:
!pip install psycopg2

In [97]:
from configparser import ConfigParser
import psycopg2

In [98]:
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="postgres")

In [101]:
def config(filename='../Postgresql/postgresDB.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [175]:

def createDict(infoTuple, columnList, index):
    infoList = []
    for i in infoTuple:
        infoList.append(i)
        if (infoTuple.index(i) == index):
            break;
    return(dict(zip(columnList, infoList)))

fullUserDict= {}
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        print('Retrieving data:')
        print('--------------')
        #Retrieving data
        cur.execute('SELECT * from users where user_id = '+str(405))
        user = cur.fetchone()
        cur.execute('SELECT role from roles where role_id = '+str(user[6]))
        role = cur.fetchone()
        # Saving the user as a dict
        userTableColumns = ['user_id', 'firstName', 'lastName', 'age', 'email', 'role']
        userDict = createDict(user, userTableColumns, 4)
        userDict["role"] = role[0]
        print(userDict)
        print('--------------')
        
        cur.execute('SELECT address_id from users where user_id = '+str(405))
        address_id = cur.fetchone()
        cur.execute('SELECT * from address where address_id = '+str(address_id[0]))
        address = cur.fetchone()
        zipcode = address[2]
        cur.execute('SELECT * from cities where zipcode = '+str(zipcode))
        city = cur.fetchone()
        addressList = list(address)
        addressList.append(city[1])
        
        addressTableColumns = ['address_id', 'address', 'zipcode', 'city']
        addressDict = createDict(addressList, addressTableColumns, 3)
        print(addressDict)
        print('--------------')
        
        fullUserDict = userDict
        fullUserDict['address'] = addressDict
        
        
        # display the PostgreSQL database server version
        #db_version = cur.fetchone()
        #print(db_version)
       
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
            #print(fullUserDict)
            return fullUserDict
            


if __name__ == '__main__':
    connect()
    
fullUserDict = connect()
print(fullUserDict)

Connecting to the PostgreSQL database...
Retrieving data:
--------------
{'user_id': 405, 'firstName': 'Belinda', 'lastName': 'Foster', 'age': 44, 'email': 'b.foster@randatmail.com', 'role': 'user'}
--------------
{'address_id': 9, 'address': 'Kongshøj Allé 65', 'zipcode': 4220, 'city': 'Korsør'}
--------------
Database connection closed.
Connecting to the PostgreSQL database...
Retrieving data:
--------------
{'user_id': 405, 'firstName': 'Belinda', 'lastName': 'Foster', 'age': 44, 'email': 'b.foster@randatmail.com', 'role': 'user'}
--------------
{'address_id': 9, 'address': 'Kongshøj Allé 65', 'zipcode': 4220, 'city': 'Korsør'}
--------------
Database connection closed.
{'user_id': 405, 'firstName': 'Belinda', 'lastName': 'Foster', 'age': 44, 'email': 'b.foster@randatmail.com', 'role': 'user', 'address': {'address_id': 9, 'address': 'Kongshøj Allé 65', 'zipcode': 4220, 'city': 'Korsør'}}


#### MongoDB

In [ ]:
!pip install pymongo

In [176]:
import pymongo
import bson
import pickle
import base64

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

#use database named "organisation"
mydb = myclient["ExamProject"]

#use collection named "developers"
mycol = mydb["Temporary"]

In [177]:
def getData(n1, n2):
    dataList = []
    tic = time.perf_counter()
    for n in range(int(n1), int(n2)+1):
        #print(n)
        #print(r.hgetall(str(n)))
        d = r.get(n)
        yourdict = pickle.loads(d)
        dataList.append(yourdict)
    toc = time.perf_counter()
    dataList.append(toc - tic)
    return dataList

car = getData(2,2)
car[0]

{'Year': 2017,
 'Mileage': 15630,
 'City': 'Katy',
 'State': ' TX',
 'Vin': '1FMJU1JT4HEA55066',
 'Make': 'Ford',
 'Model': 'ExpeditionXLT'}

In [179]:
booking = {}
booking['user'] = fullUserDict
booking['car'] = car[0]

print(booking)

{'user': {'user_id': 405, 'firstName': 'Belinda', 'lastName': 'Foster', 'age': 44, 'email': 'b.foster@randatmail.com', 'role': 'user', 'address': {'address_id': 9, 'address': 'Kongshøj Allé 65', 'zipcode': 4220, 'city': 'Korsør'}}, 'car': {'Year': 2017, 'Mileage': 15630, 'City': 'Katy', 'State': ' TX', 'Vin': '1FMJU1JT4HEA55066', 'Make': 'Ford', 'Model': 'ExpeditionXLT'}}


In [180]:
x = mycol.insert_one(booking)

In [181]:
for x in mycol.find():
    print(x)

{'_id': ObjectId('628d2b1beedb38463899a41e'), 'Year': 2016, 'Mileage': 24925, 'City': 'Tucson', 'State': ' AZ', 'Vin': '1HGCR2F82GA002625', 'Make': 'Honda', 'Model': 'Accord'}
{'_id': ObjectId('628d2b6ceedb38463899a41f'), 'Year': 2017, 'Mileage': 15630, 'City': 'Katy', 'State': ' TX', 'Vin': '1FMJU1JT4HEA55066', 'Make': 'Ford', 'Model': 'ExpeditionXLT'}
{'_id': ObjectId('628dbb5deedb38463899a421'), 'user': {'user_id': 405, 'firstName': 'Belinda', 'lastName': 'Foster', 'age': 44, 'email': 'b.foster@randatmail.com', 'role': 'user', 'address': {'address_id': 9, 'address': 'Kongshøj Allé 65', 'zipcode': 4220, 'city': 'Korsør'}}, 'car': {'Year': 2017, 'Mileage': 15630, 'City': 'Katy', 'State': ' TX', 'Vin': '1FMJU1JT4HEA55066', 'Make': 'Ford', 'Model': 'ExpeditionXLT'}}


In [ ]:
%%writefile application.py

from flask import Flask, request, jsonify, render_template
import redis
import json
import time
import pandas as pd

app = Flask(__name__, template_folder='hmtl_documents')

# ---------------- THE CODE -------------------------
df_carList = pd.read_csv("./datasets/Car_listings.csv", delimiter=",")
df_carList = df_carList.drop(['Price'], axis = 1)

df_carList.to_json('./datasets/cars.json', orient='split', compression='infer', index='true')
df = pd.read_json('./datasets/cars.json', orient ='split', compression = 'infer')

r = redis.Redis(db=2)

with open('./datasets/cars.json') as json_file:
    data = json.load(json_file)
    
# Loop og give hver værdi en key som er deres kolonne navn
columns = list(df)
listOfDict = []

def smallDictFunction(co: [], di: {}) -> {}:
    smallDicts = {}
    for c in co:
        for d in di:
            if (columns.index(c) == dic.index(d)):
                smallDicts[c] = d
    return smallDicts
   
for dic in data['data']:
    listOfDict.append(smallDictFunction(columns, dic))
    if (data['data'].index(dic)+1 == 5):
        break    # break here

index_list = [*range(1, (len(listOfDict)+1), 1)]
fullDict = dict(zip(index_list, listOfDict))

with open("./datasets/carsDict.json", "w") as write_file:
    json.dump(fullDict, write_file, indent=4)

with open('./datasets/carsDict.json') as json_file:
    data = json.load(json_file)

with r.pipeline() as pipe:
    for dic in data:
        #print(dic)
        #print(data[dic])
        pipe.hmset(dic, data[dic])
        pipe.execute()
r.bgsave()

def getData(n1, n2):
    dataList = []
    tic = time.perf_counter()
    for n in range(int(n1), int(n2)+1):
        #print(n)
        #print(r.hgetall(str(n)))
        d = r.hgetall(str(n))
        dataList.append(d)
    toc = time.perf_counter()
    dataList.append(toc - tic)
    return dataList

def getAttributes():
    tic = time.perf_counter()
    unPrettyList = list(r.hgetall('1'))
    toc = time.perf_counter()
    prettyList = []
    for s in unPrettyList:
        txt = str(s).split("'")
        prettyList.append(txt[1])
    time2 = toc - tic
    prettyList.append(time2)
    return prettyList

def getBySpecificAtt(n, s):
    tic = time.perf_counter()
    string = r.hget(n,s)
    toc = time.perf_counter()
    time2 = toc - tic
    txt = str(string).split("'")
    
    dataList = []
    dataList.append(txt[1])
    dataList.append(time2)
    return dataList


# ---------------- THE ENDPOINTS --------------------
@app.route('/')
def start():
    attributes = getAttributes()
    
    return render_template('start.html', attributes=attributes[1:-1], time=attributes[-1])

@app.route('/data', methods=['GET', 'POST'])
def data():
    if request.method == 'POST':
        n1 = request.form['n1']
        n2 = request.form['n2']
        data = getData(n1, n2)
          
        return render_template("data.html", data=data[int(n1):int(n2)], time=data[-1])
    
@app.route('/specificData', methods=['GET', 'POST'])
def specificData():
    if request.method == 'POST':
        n = request.form['n']
        s = request.form['s']
        data = getBySpecificAtt(n, s)
          
        return render_template("specificData.html", data=data[0], time=data[1])

@app.route('/redis', methods=['GET', 'POST'])
def predict():
    if request.method == 'GET':
        data = request.data
    return render_template('redis.html', carsDict=data)

if __name__ == '__main__':
    app.run(debug=True)

http://localhost:5000/

In [ ]:
!python application.py